In [1]:
import pandas as pd
import altair as alt
# alt.renderers.enable("default")
alt.data_transformers.disable_max_rows()
import warnings
warnings.filterwarnings('ignore')

# import sys
# sys.path.append('../../')
# from script.utils import save_chart

In [2]:
alt.__version__

'4.2.0'

In [3]:
nn_repo_df = pd.read_csv("../../script/data/csvs/github_data/network_navigator_repo.csv")
nn_commits_df = pd.read_csv("../../script/data/csvs/github_data/network_navigator_commits.csv")

In [4]:
nn_commits_df['commit.author.date'] = pd.to_datetime(nn_commits_df['commit.author.date'], errors='coerce')

In [5]:
nn_commits_df.loc[nn_commits_df['commit.author.name'] == 'Zoe LeBlanc', 'commit.author.name'] = 'ZoeLeBlanc'

In [6]:
nn_commits_df['commit.author.name'].value_counts()

JR Ladd            171
ZoeLeBlanc          45
Matthew Lincoln      1
Name: commit.author.name, dtype: int64

In [7]:
subset_commits_df = nn_commits_df[['commit.author.name', 'commit.author.date', 'stats.total', 'stats.additions', 'stats.deletions']]

cols = subset_commits_df.columns
cols = [col.replace('.', '_') for col in cols]
subset_commits_df.columns = cols

# Prepare data for symmetric bar chart
df_additions = subset_commits_df[["commit_author_name", "commit_author_date", "stats_additions"]].copy()
df_additions["type"] = "additions"
df_additions.rename(columns={"stats_additions": "value"}, inplace=True)

df_deletions = subset_commits_df[["commit_author_name", "commit_author_date", "stats_deletions"]].copy()
df_deletions["type"] = "deletions"
df_deletions["stats_deletions"] = -df_deletions["stats_deletions"]
df_deletions.rename(columns={"stats_deletions": "value"}, inplace=True)

df_graph = pd.concat([df_additions, df_deletions])


In [8]:
# Create the symmetric bar chart
project_history = alt.Chart(df_graph).mark_bar().encode(
    x=alt.X('commit_author_date:T', title="Date"),
    y=alt.Y('value:Q', title="Additions/Deletions"),
    color=alt.Color('type:N', title="Type of Change"),
    tooltip=['commit_author_name:N', 'commit_author_date:T', 'value:Q', 'type:N'],
).properties(
    title="Additions and Deletions to Network Navigator",
)

project_history

alt.Chart(...)

In [10]:
selection = alt.selection_point(fields=['commit_author_name'], bind='legend')

author_chart = alt.Chart(subset_commits_df).mark_bar(opacity=0.7).encode(
    x=alt.X('commit_author_date:T', title='Date'),
    y=alt.Y('stats_total:Q', title='Total Contributions').stack(None),
    color=alt.Color('commit_author_name:N',
        scale=alt.Scale(scheme='tableau20'), 
        legend=alt.Legend(title="Author"),
        sort=alt.EncodingSortField(
            field="stats_total",  # The field to use for the sort
            op="sum",            # The operation to run on the field prior to sorting
            order="descending"   # The sort order
        )
    ),
    tooltip=['commit_author_name:N', 'commit_author_date:T', 'stats_total:Q'],
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
).add_params(selection).interactive(bind_y=False).properties(
    title="Contributors to Network Navigator",
)

author_chart


AttributeError: module 'altair' has no attribute 'selection_point'

In [11]:
nn_chart = alt.hconcat(project_history, author_chart).resolve_scale(y='independent', color='independent').properties(
    title="History of Network Navigator via GitHub").configure(
        title=alt.TitleConfig(fontSize=20, font='Garamond', anchor='middle'),
        axis=alt.AxisConfig(labelFont='Garamond', titleFont='Garamond'),
        legend=alt.LegendConfig(labelFont='Garamond', titleFont='Garamond')
    )

nn_chart

NameError: name 'author_chart' is not defined

In [100]:
save_chart(nn_chart, "../../media/project_history.png", scale_factor=2.0)